## Beam/Correlation Functions

### This notebook contains the functions/actions needed to create beam maps and a correlation matrix for mosaicking.  

##### June 6, 2019 D.J. Pisano

In [1]:
# Import needed packages
import os;
import shutil
import glob
import apercal.libs.lib as lib
import apercal
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from astropy.time import Time
import subprocess

myusername = os.environ['USER']
if not ('PYTHONPATH' in os.environ and
        '/home/' + myusername + '/apercal' in os.environ['PYTHONPATH']):
      print("WARNING: your apercal directory should be in the $PYTHONPATH")

%config IPCompleter.greedy=True
%matplotlib notebook
lib.setup_logger('info', logfile='/home/{}/testing/logs/mosaic.log'.format(myusername))

cfgfile='/home/{}/testing/cfg/mosaic.cfg'.format(myusername)

metadir='/home/{}/metadata/'.format(myusername)   # This directory is where the beam map and correlation matrix reside.

%matplotlib inline



root - INFO : Logging started!
root - INFO : To see the log in a bash window use the following command:
root - INFO : tail -n +1 -f /home/pisano/testing/logs/mosaic.log


In [2]:
# Get needed values from configuration file for where data is and where it is going.
prepare=apercal.prepare(cfgfile)

basedir=prepare.basedir
projectid=prepare.prepare_date+prepare.prepare_obsnum_target

linedir=basedir+projectid+'/line'
continuumdir=basedir+projectid+'/continuum/'

imagedir=linedir+'/raw/'

mosaicdir=linedir+'/'+prepare.mossubdir+'/'
beamdir=basedir+projectid+'/beams/'

# Create directories as needed.
if os.path.isdir(basedir+projectid)==False:
    os.mkdir(basedir+projectid)
if os.path.isdir(linedir)==False:
    os.mkdir(linedir)
if os.path.isdir(continuumdir)==False:
    os.mkdir(continuumdir)
if os.path.isdir(imagedir)==False:
    os.mkdir(imagedir)
if os.path.isdir(mosaicdir)==False:
    os.mkdir(mosaicdir)
if os.path.isdir(beamdir)==False:
    os.mkdir(beamdir)


config - INFO :  Configuration file /home/pisano/testing/cfg/mosaic.cfg successfully read!


In [3]:
# Functions needed to create Gaussian beams (with appropriate cutoff)

def create_beam(bm_num):
    # This will create a default Gaussian beam using "imgen" in Miriad
    # For these purposes, the beam will be assumed to be Gaussian with a FWHM=33.7 arcmin
    bm_size=4097     # Number of pixels for beam map
    pos=[0.,60.]      # Set reference pixel to RA=0, Dec=60 (arbitrary)
    pixel=4.         # Cell size in arcsec
    pk=1.            # Peak of Beam response
    fwhm=32.5*60.    # FWHM of WSRT beam in arcsec (very rough)
    bpa=0.           # PA of beam 
    # create beam
    imgen = lib.miriad('imgen')
    imgen.out = beamdir+'beam_{}o.map'.format(str(bm_num).zfill(2))
    imgen.imsize = bm_size
    imgen.cell = pixel
    imgen.object = 'gaussian'
    imgen.spar = '{0},{1},{2},{3},{4},{5}'.format(str(pk),str(0),str(0),str(fwhm),str(fwhm),str(bpa))
    imgen.radec = '{0},{1}'.format(str(pos[0]),str(pos[1]))
    imgen.inp()
    imgen.go()

def beam_cutoff(bm_num):
    maths = lib.miriad('maths')
    maths.exp="'<"+beamdir+"beam_{}o.map>'".format(str(bm_num).zfill(2))
    maths.mask="'<"+beamdir+"beam_{}o.map>.gt.0.25'".format(str(bm_num).zfill(2))
    maths.out=beamdir+"beam_{}.map".format(str(bm_num).zfill(2))
    maths.inp()
    maths.go()
    os.system('rm -rf '+beamdir+'beam_{}o.map'.format(str(bm_num).zfill(2)))
    
def fixheader(bm_num):
    # When created, files have the Gaussian parameters included as the "beam".  
    # This function removes those values.
    delhd = lib.miriad('delhd')
    delhd.in_ = beamdir+'beam_{}.map/bmaj'.format(str(bm_num).zfill(2))
    delhd.go()
    delhd.in_ = beamdir+'beam_{}.map/bmin'.format(str(bm_num).zfill(2))
    delhd.go()
    delhd.in_ = beamdir+'beam_{}.map/bpa'.format(str(bm_num).zfill(2))
    delhd.go()
        
    
def write_fits(infile):
    # This function will take an input Miriad file name and write out a FITS file
    # Then it will delete the Miriad file
    fits = lib.miriad('fits')
    fits.op = 'xyout'
    fits.in_ = infile
    fits.out = infile.replace('map','fits')
    fits.inp()
    fits.go()
    os.system('rm -rf {}'.format(infile))


In [ ]:
# Create 40 beams (00-39):

for i in range(40):
    create_beam(i)
    beam_cutoff(i)
    fixheader(i)
    write_fits(beamdir+'beam_{}.map'.format(str(i).zfill(2)))
    

In [5]:
# Create one master beam:

i=0
create_beam(i)
beam_cutoff(i)
fixheader(i)
write_fits(beamdir+'beam_{}.map'.format(str(i).zfill(2)))
os.rename(beamdir+'beam_{}.fits'.format(str(i).zfill(2)),metadir+'gauss_beam.fits')

miriad imgen - INFO : task=imgen, object=gaussian, cell=4.0, radec=0.0,60.0, spar=1.0,0,0,1950.0,1950.0,0.0, out=/data4/pisano/190428055/beams/beam_00o.map, imsize=4097
miriad maths - INFO : task=maths, mask='</data4/pisano/190428055/beams/beam_00o.map>.gt.0.25', exp='</data4/pisano/190428055/beams/beam_00o.map>', out=/data4/pisano/190428055/beams/beam_00.map
miriad fits - INFO : task=fits, out=/data4/pisano/190428055/beams/beam_00.fits, in_=/data4/pisano/190428055/beams/beam_00.map, op=xyout


OSError: [Errno 18] Invalid cross-device link

In [6]:
# Function to create a correlation matrix

def symmetrize(a):
    return a + a.T - np.diag(a.diagonal())

def create_correlation():
    # This function creates a correlation matrix for 39 independent beams (i.e. an identity matrix)
    # And writes it out to a file
    # This needs to be multiplied by the variance for each beam still.
    C=np.identity(40,dtype=float)
    # Fill array with estimated correlation coefficients (r in the nomenclature)
    # These are eyeballed based on ASKAP measurements.
    C[0,17]=0.7
    C[0,24]=0.7
    C[0,18]=0.25
    C[0,23]=0.25
    C[1,2]=0.11
    C[1,8]=0.11
    C[2,3]=0.11
    C[2,8]=0.11
    C[2,9]=0.11
    C[3,4]=0.11
    C[3,9]=0.11
    C[3,10]=0.11
    C[4,5]=0.11
    C[4,10]=0.11
    C[4,11]=0.11
    C[5,6]=0.11
    C[5,11]=0.11
    C[5,12]=0.11
    C[6,7]=0.11
    C[6,12]=0.11
    C[6,13]=0.11
    C[7,13]=0.11
    C[7,14]=0.11
    C[8,9]=0.11
    C[8,15]=0.11
    C[9,10]=0.11
    C[9,15]=0.11
    C[9,16]=0.11
    C[10,11]=0.11
    C[10,16]=0.11
    C[10,17]=0.11
    C[11,12]=0.11
    C[11,17]=0.11
    C[11,18]=0.11
    C[12,13]=0.11
    C[12,18]=0.11
    C[12,19]=0.11
    C[13,14]=0.11
    C[13,19]=0.11
    C[13,20]=0.11
    C[14,20]=0.11
    C[15,16]=0.11
    C[15,21]=0.11
    C[15,22]=0.11
    C[16,17]=0.11
    C[16,22]=0.11
    C[16,23]=0.11
    C[17,18]=0.11
    C[17,23]=0.11
    C[17,24]=0.11
    C[18,19]=0.11
    C[18,24]=0.11
    C[18,25]=0.11
    C[19,20]=0.11
    C[19,25]=0.11
    C[19,26]=0.11
    C[20,26]=0.11
    C[21,22]=0.11
    C[21,27]=0.11
    C[22,23]=0.11
    C[22,27]=0.11
    C[22,28]=0.11
    C[23,24]=0.11
    C[23,28]=0.11
    C[23,29]=0.11
    C[24,25]=0.11
    C[24,29]=0.11
    C[24,30]=0.11
    C[25,26]=0.11
    C[25,30]=0.11
    C[25,31]=0.11
    C[26,31]=0.11
    C[26,32]=0.11
    C[27,28]=0.11
    C[27,33]=0.11
    C[27,34]=0.11
    C[28,29]=0.11
    C[28,34]=0.11
    C[28,35]=0.11
    C[29,30]=0.11
    C[29,35]=0.11
    C[29,36]=0.11
    C[30,31]=0.11
    C[30,36]=0.11
    C[30,37]=0.11
    C[31,32]=0.11
    C[31,37]=0.11
    C[31,38]=0.11
    C[32,38]=0.11
    C[32,39]=0.11
    C[33,34]=0.11
    C[34,35]=0.11
    C[35,36]=0.11
    C[36,37]=0.11
    C[37,38]=0.11
    C[38,39]=0.11
    C=symmetrize(C)
    np.savetxt(metadir+'correlation.txt',C,fmt='%f')


In [7]:
# Create correlation matrix
create_correlation()